<a href="https://colab.research.google.com/github/LucianaNieto/CarbonSequestration/blob/main/SOC_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


----
Objectives:



### Libraries

In [2]:
!pip install -q pycountry
!pip install -q plotly --upgrade
!pip install -q -U kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.3 MB/s eta 0:00:00


In [54]:
#Libraries
import pandas as pd
import numpy as np
import os
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.formula.api import ols
import statsmodels.api as sm
import matplotlib.pyplot as plt

### Directories

In [4]:

# base path
base_path = '/content/drive/My Drive/2024/DataScience/CarbonSequestration/01-CarbonSequestration/'

# Define paths for data and figure subdirectories
paths = {
    'raw_data_path': base_path + '01-Data/01-RawData/',
    'processed_data_path': base_path + '01-Data/02-ProcessedData/',
    'exploratory_figures_path': base_path + '02-Figures/01-Exploratory/',
    'final_figures_path': base_path + '02-Figures/02-Finals/',
    'raw_code_path': base_path + '03-Code/01-RawCode/',
    'data_processing_code_path': base_path + '03-Code/02-DataProcessing/',
    'data_analysis_code_path': base_path + '03-Code/03-DataAnalysis/',
    'clean_scripts_path': base_path + '03-Code/04-CleanScripts/'
}

### Data loading

In [6]:
def load_data(file_name):
    csv_path = os.path.join(paths['processed_data_path'], f"{file_name}.csv")
    return pd.read_csv(csv_path)

df_effects = load_data('df_effects_SOC_filtered')
print("\nEffect-sizes Columns and Shape:")
print(df_effects.columns)
print(df_effects.shape)



Effect-sizes Columns and Shape:
Index(['id', 'land_use', 'intervention', 'sub_cat_intervention', 'details',
       'control (c)', 'treatment_authors', 'summary_effect_size', 'management',
       'method', 'species', 'region_climate', 'soil', 'depth_original',
       'depth', 'group_depth', 'depth2', 'duration', 'carbon', 'unit',
       'rateyr', 'log_scale', 'metric', 'outcome', 'sub_cat_outcome',
       'details_outcome', 'lower_ci', 'effect size', 'upper_ci', 'p_value',
       'n_paired_data', 'es_se', 'duration_standardized'],
      dtype='object')
(5164, 33)


### Modeling

In [48]:
custom_color_scale = ['#e3fef7','#003c43']
# Convert columns to numeric
numeric_cols = ['depth', 'duration_standardized']
for col in numeric_cols:
    df_effects[col] = pd.to_numeric(df_effects[col], errors='coerce')

# Fill missing values for numeric columns
df_effects[numeric_cols] = df_effects[numeric_cols].fillna(df_effects[numeric_cols].mean())

# Fill missing values for categorical columns
categorical_cols = ['land_use']
df_effects[categorical_cols] = df_effects[categorical_cols].fillna('Unknown')

# ANOVA for effect size by land use
anova_model = ols('Q("effect size") ~ C(land_use)', data=df_effects).fit()
anova_table = sm.stats.anova_lm(anova_model, typ=2)
print(anova_table)

# Boxplot of effect size by land use
fig = px.box(df_effects, x='land_use', y='effect size',
             title='Effect Size by Land Use',
             labels={'land_use': 'Land Use', 'effect size': 'Effect Size'},
             color_discrete_sequence=[custom_color_scale[0]])
fig.update_layout(
    plot_bgcolor='#003c43',
    paper_bgcolor='#003c43',
    title_font=dict(color=custom_color_scale[0]),
    font=dict(color=custom_color_scale[0]),
    xaxis=dict(title_font=dict(color=custom_color_scale[0]), tickfont=dict(color=custom_color_scale[0])),
    yaxis=dict(title_font=dict(color=custom_color_scale[0]), tickfont=dict(color=custom_color_scale[0]))
)

fig.show()


                  sum_sq      df           F         PR(>F)
C(land_use)   242.642214    10.0  102.627693  8.363162e-195
Residual     1218.321574  5153.0         NaN            NaN


In [52]:
# Tukey's HSD test
tukey = pairwise_tukeyhsd(endog=df_effects['effect size'], groups=df_effects['land_use'], alpha=0.05)
print(tukey.summary())

# Extract Tukey HSD results
tukey_results = pd.DataFrame(data=tukey._results_table.data[1:], columns=tukey._results_table.data[0])

# Plot the results with Plotly
fig = go.Figure()

for i in range(len(tukey_results)):
    row = tukey_results.iloc[i]
    fig.add_trace(go.Scatter(
        x=[row['lower'], row['upper']],
        y=[row['group1'] + ' vs ' + row['group2']] * 2,
        mode='lines',
        line=dict(color='#b0e0e6', width=2),
        showlegend=False
    ))
    fig.add_trace(go.Scatter(
        x=[(row['meandiff'])],
        y=[row['group1'] + ' vs ' + row['group2']],
        mode='markers',
        marker=dict(color='#b0e0e6', size=10),
        showlegend=False
    ))

fig.update_layout(
    title='Tukey HSD Test Results',
    xaxis_title='Mean Difference',
    yaxis_title='Group Comparison',
    plot_bgcolor='#003c43',
    paper_bgcolor='#003c43',
    title_font=dict(color=custom_color_scale[0]),
    font=dict(color=custom_color_scale[0]),
    xaxis=dict(title_font=dict(color=custom_color_scale[0]), tickfont=dict(color=custom_color_scale[0]), showgrid=False),
    yaxis=dict(title_font=dict(color=custom_color_scale[0]), tickfont=dict(color=custom_color_scale[0]), showgrid=False)
)

fig.show()

            Multiple Comparison of Means - Tukey HSD, FWER=0.05             
      group1             group2       meandiff p-adj   lower   upper  reject
----------------------------------------------------------------------------
          cropland             desert  -0.3486 0.8072 -0.9888  0.2915  False
          cropland        forest land  -0.3113    0.0 -0.3798 -0.2429   True
          cropland          grassland  -0.6346    0.0   -0.72 -0.5491   True
          cropland    land_use change  -0.2256    0.0 -0.2848 -0.1663   True
          cropland other land:various  -0.9803 0.0004 -1.6814 -0.2792   True
          cropland         shrublands  -0.1839  0.682 -0.4873  0.1194  False
          cropland             tundra  -0.5127 0.0211 -0.9861 -0.0394   True
          cropland            various  -1.1294 0.0002 -1.9131 -0.3458   True
          cropland  various land uses  -0.4112    0.0 -0.4746 -0.3479   True
          cropland           wetlands  -1.1711    0.0 -1.4459 -0.8963   True

In [26]:

# Interaction effect model
interaction_model = ols('Q("effect size") ~ C(land_use) * depth + C(land_use) * duration_standardized', data=df_effects).fit()
interaction_anova_table = sm.stats.anova_lm(interaction_model, typ=2)
print(interaction_anova_table)


                                        sum_sq      df           F  \
C(land_use)                         286.256934    10.0  122.528447   
depth                                 0.313509     1.0    1.341935   
C(land_use):depth                    13.466928    10.0    5.764338   
duration_standardized                 0.188923     1.0    0.808661   
C(land_use):duration_standardized    24.996776    10.0   10.699535   
Residual                           1200.130996  5137.0         NaN   

                                          PR(>F)  
C(land_use)                        1.823104e-188  
depth                               2.467468e-01  
C(land_use):depth                   1.137253e-08  
duration_standardized               3.685587e-01  
C(land_use):duration_standardized   3.443269e-18  
Residual                                     NaN  


/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning:

covariance of constraints does not have full rank. The number of constraints is 10, but rank is 8



----
----

**Results**

The analysis of variance (ANOVA) and Tukey's Honest Significant Difference (HSD) test were conducted to evaluate the effect of land use on effect size. The ANOVA results revealed that land use significantly affects effect size (F(10, 5153) = 102.63, p < 0.001). Additionally, significant interactions were observed between land use and depth (F(10, 5137) = 5.76, p < 0.001) and between land use and standardized duration (F(10, 5137) = 10.70, p < 0.001).

The Tukey HSD test further elucidated these relationships by identifying specific pairwise differences between land use categories. Significant differences in effect size were found between various land use types, notably between cropland and forest land (mean difference = -0.3113, p < 0.001), cropland and grassland (mean difference = -0.6346, p < 0.001), and cropland and wetlands (mean difference = -1.1711, p < 0.001). These results indicate that cropland consistently exhibits a lower effect size compared to these other land use categories.

The interaction model incorporating land use, depth, and standardized duration demonstrated that both depth and standardized duration significantly interact with land use to influence effect size. Specifically, the interactions between land use and depth (F(10, 5137) = 5.76, p < 0.001) and between land use and standardized duration (F(10, 5137) = 10.70, p < 0.001) underscore the complexity of these relationships.

**Discussion**

The findings from this study underscore the pivotal role of land use in determining effect size, with cropland exhibiting a notably lower effect size compared to other land use types such as forest land, grassland, and wetlands. The significant interactions between land use and both depth and standardized duration suggest that the effect of land use on effect size is modulated by these factors.

The ANOVA results provide robust evidence that land use is a critical determinant of effect size, accounting for a substantial portion of the variance. The Tukey HSD test complements this by specifying the pairwise differences, highlighting the particularly low effect size associated with cropland. This is consistent with the hypothesis that different land use practices impact ecological outcomes differently, with cropland potentially leading to more degraded or less favorable conditions.

The interaction effects identified in the model suggest that the impact of land use on effect size cannot be fully understood without considering the modifying effects of depth and standardized duration. This indicates that land management practices need to be tailored not only to the type of land use but also to specific conditions such as soil depth and the duration of land use practices.

In conclusion, this study provides compelling evidence that land use, depth, and duration are interlinked factors influencing ecological outcomes, as measured by effect size. Future research should focus on elucidating the mechanisms driving these interactions and on developing land management strategies that mitigate negative impacts while enhancing positive outcomes in diverse ecological contexts.